
      
      
   
# G4 - Adversarial Training 2
This notebook presents the reproducibility report of our group, where we have two sections each dealing with one of the papers respectively.    
We will introduce the basic concepts and frameworks of the paper before showing their empirical findings and our attempts to reproduce them.

## First paper: More Data Can Expand the Generalization Gap Between Adversarially Robust and Standard Models - *Lin Chen, Yifei Min, Mingrui Zhang, Amin Karbasi*

The first paper deals with the challenge of decreasing the performance gap on natural (non-perturbed) datasets between standard machine learning models and adversarially trained models. Its main point is that the naive approach of simply increasing the training dataset size may not always help in decreasing this gap, in fact it might increase it depending on the strength of the adversary.

### General setting
The paper examines the 'cross-generalization gap' (as they call it) under two settings:


1.   Binary Classification with Gauss and Bernoulli distribution
2.   Linear Regression with Gauss and Poisson distribution

In the both cases, we have the following problem setup:

![Points](src/cross_gen_gap/graphics/img.png)




### Binary Classification
In Binary Classification, we have datapoints $(x,y) \in \mathbb{R}^{d} \times\{\pm 1\}$, model parameter $w \in \mathbb{R}^{d}$ and loss function $\ell(x, y ; w)=-y\langle w, x\rangle$. This means that the paper limits itself to the case of a linear classifier, in which the model is further restricted by $\|w\|_{\infty} \leq W$, where $W$ is some positive real number.   


#### Gaussian Model
In the Gaussian Model, the datapoints for training and test datasets are generated by a Gaussian distribution, i.e. $y\sim$ Unif$(\{\pm 1\})$ and $x \mid y \sim \mathcal{N}(y \mu, \Sigma)$, where $\mu(j) \geq 0$ for $\forall j \in[d]$ and $\Sigma=\operatorname{diag}\left(\sigma(1)^{2}, \sigma(2)^{2}, \ldots, \sigma(d)^{2}\right)$.     
For the empirical studies, the paper simplifies the model by setting $W = d = \mu = 1$ and $ \sigma = 2$, in other words it only considers a 1-dimensional case with class centers around 1 and -1 respectively, where the weight $w\in \mathbb{R}$ of the model is bounded by $\|w\|_{\infty} \leq 1$.   
To study the cross generalization gap, the authors investigate the performance (test loss) of this model with different values for $\varepsilon$ and increasing training data sizes $n$ for each of the inspected values of $\varepsilon$. The graph below visualizes these empirical studies.    

The main finding of the paper in this section is that 
1. if the adversarial strength $\varepsilon$ is small enough (precise bounds stated in the paper), then there are two stages of the cross generalization gap for increasing training dataset sizes $n$: an initial strictly increasing stage followed by a stage where the gap is strictly decreasing
2. if the strength of the adversary is too large, then the cross generalization gap will be strictly increasing for all $n \in \mathbb{N}$



Loss | Generalization Gap
-|-
![Loss](src/cross_gen_gap/graphics/loss_values_gauss.png) | ![Loss](src/cross_gen_gap/graphics/gen_gap_gauss.png)

<!---
<tr>
<td> ![Loss](src/cross_gen_gap/graphics/loss_values_gauss.png) </td>
<td> <img src="src/cross_gen_gap/graphics/gen_gap_gauss.png" alt="Cross generalization gap between models trained on adversarially perturbed datasets and standard model" style="width: 380px;"/> </td>
</tr>
--->

Both plots shown above are taken directly from the paper and represent empirical findings we aim to reproduce here.
The right figure directly follows from the left one if we simply take the difference between each of the $\varepsilon$ models and the standard model, respectively. As the loss function and cross generalization gap is defined over expected values, these curves show averages over multiple runs of the training and evaluation process. The paper doesn't state how often they rerun the experiments and if they used some smoothening in plotting the graphs. 

Now, let's try to reproduce this!

In [8]:
# If this notebook is being run on Google Colab, uncomment the following lines
# In order for this to work, create a folder g4-adv-training on drive and paste the repo contents there
"""

from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/g4-adv-training'

# This will mount google drive under 'MyDrive'
drive.mount('/content/gdrive', force_remount=True)
# Navigate to our base folder
os.chdir(gdrive_path)
"""

"\n\nfrom google.colab import drive\nimport os\n\ngdrive_path='/content/gdrive/MyDrive/g4-adv-training'\n\n# This will mount google drive under 'MyDrive'\ndrive.mount('/content/gdrive', force_remount=True)\n# Navigate to our base folder\nos.chdir(gdrive_path)\n"

In [10]:
# Import useful libraries and own code
import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn import datasets
import pandas as pd
from src.cross_gen_gap.bin_classification import *

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# by uncommenting the lines below, you can try out our setup with different values for eps
# and number of repititions; for setting below, this takes about 2min but will be very inaccurate
"""

eps_list = [0.0, 0.5, 1.0]
n_points = list(range(1, 21,4))
adv_test_loss = train_classification_model(eps_list, num_repetition = 50, n_points = n_points, points_step_size=4)
plot_class_loss(eps_list, adv_test_loss, training_points=n_points)
plot_class_gap(eps_list, adv_test_loss, training_points=n_points)
"""

We have the following experiment setup (please have a look at the file `src/cross_gen_gap/bin_classification.py` for reference):
* We adversarially train a model for a particular value of $\varepsilon$ on a dataset of size n where n ranges from 1 to 20
* We test this model on a test dataset of size 5000 and save the loss

We repeat this experiment 100 times for each chosen value of the adversarial strength and average over the resulting loss values. This experiment is conducted for a Gaussian data sampling and for Bernoulli data sampling.

# Plot (Gaussian)


<!---
<tr>
<tr>
<td> <img src="src/cross_gen_gap/graphics/loss_values_gauss.png" alt="Loss values for different training sizes and epsilon values" style="width: 450px;"/> </td>  
<td> <img src="src/cross_gen_gap/graphics/class_loss_gauss_smooth.png" alt="Loss values for different training sizes and epsilon values" style="width: 400px;"/> </td>
</tr>

<tr>
<td> <img src="src/cross_gen_gap/graphics/gen_gap_gauss.png" alt="Cross generalization gap between models trained on adversarially perturbed datasets and standard model" style="width: 380px;"/> </td>
<td> <img src="src/cross_gen_gap/graphics/generalization_gap_bin_class_gauss-1.png" alt="Cross generalization gap between models trained on adversarially perturbed datasets and standard model" style="width: 400px;"/> </td>
</tr>
--->

Loss from the paper | Our loss values
-|-
![Loss](src/cross_gen_gap/graphics/loss_values_gauss.png) | ![Loss](src/cross_gen_gap/graphics/class_loss_gauss_smooth.png)
**Generalizetion Gap from the paper** | **Our generalization gap**
![Loss](src/cross_gen_gap/graphics/gen_gap_gauss.png) | ![Loss](src/cross_gen_gap/graphics/generalization_gap_bin_class_gauss-1.png)


The plots above show our results for the case of Gaussian model classification. 
We see that our test loss for the standard model converges to -1 as the training set size increases.
For most smaller $\varepsilon$'s, we see that the generalization gap increases first and then falls resulting in a peak somewhere between the training set sizes 5 to 10. This result is consistent with the findings in the paper. However, we observe a different behavior for $\varepsilon$ ranges between 0.7 and 1, we do not see a decreasing behaviour for larger training set sizes as opposed to the paper plots. This may be due to insufficient number of experiment runs where we were bounded by the runtimes of the training process. The peak for values $\varepsilon = 0.7$ and $\varepsilon = 0.9$ can only be faintly seen for training set sizes ranging between 5 and 10.
For larger $\varepsilon$, we see that increasing the number of training points does not close the generalization gap, which is again consistent with what the paper argues.

# Plot (Bernoulli)

<!--
<tr>
<td> <img src="src/cross_gen_gap/graphics/Loss_Bernoulli_paper.png" alt="Loss values for different training sizes and epsilon values(Paper)" style="width: 450px;"/> </td>
<td> <img src="src/cross_gen_gap/graphics/Class_loss_bern.png" alt="Loss values for different training sizes and epsilon values" style="width: 380px;"/> </td>
</tr>
-->

Loss from the paper | Our loss values
-|-
![Loss](src/cross_gen_gap/graphics/Loss_Bernoulli_paper.png) | ![Loss](src/cross_gen_gap/graphics/Class_Loss_Bern.png)

The above plots represent the test loss of an adverserially trained model for different values of the signal strength parameter $\tau$. The lower values of $\tau$ (0.1 and 0.2) correspond to the strong adversary regime, while the higher ones (0.5 and 0.7) represent the weak adversary case. The value of $\varepsilon$ is fixed to 0.2 for all the experiments in this section.

The plot shown on the left is from the paper, while the one on the right shows the results obtained by us. We are able to reproduce the behavior for the most part, as we observe an initial decline in the loss for larger $\tau$'s while a steady behavior for smaller $\tau$'s. In the presentation, we had a spike around values of 140, this was due to plotting issues and did not represent any behaviour of the data, now this is fixed and the values align.
Differing from the paper, our graphs are a lot less noisy as we only check for 10 different dataset sizes as. This suffices to see the trend proposed in the paper.

# Plot (Linear Regression - Gaussian)

<!--
<tr>
<td> <img src="src/cross_gen_gap/graphics/Lin_Reg_Loss_paper.png" alt="Loss values for different training sizes and epsilon values(Paper)" style="width: 450px;"/> </td>
<td> <img src="src/cross_gen_gap/graphics/Lin_Reg_Loss_Values-1.png" alt="Loss values for different training sizes and epsilon values" style="width: 380px;"/> </td>
</tr>
-->

Loss from the paper | Our loss values
-|-
![Loss](src/cross_gen_gap/graphics/Lin_Reg_Loss_paper.png) | ![Loss](src/cross_gen_gap/graphics/Lin_Reg_Loss_Values-1.png)

The solution to the best adverserially robust model for the case of linear regression is given by the closed form expression:  
 $$ \begin{aligned} 
w_{n}^{\mathrm{rob}} &=\underset{w \in \mathbb{R}^d}{\arg \min } \frac{1}{n} \sum_{i=1}^{n} \left(|y_i - <w,x_i>| + \varepsilon\sum_{j=1}^{d}|w_j|\right)^2 \end{aligned}$$

Using this result, we emulate models for different $\varepsilon$'s using different training sets with 1 to 20 samples. The left plot showing the variation of test loss with the number of training samples for different values of $\varepsilon$ is taken from the paper while the right figure is obtained through our experiments. The trend for different $\varepsilon$'s is similar in both the graphs, except for the behavior of the standard model ($\varepsilon$ = 0) whose curve lies above the rest of the curves in the reproduced version. As argued in the paper, the test loss for the standard model should converge to 0 on incrementing the number of training samples to 20. In our case, this convergence is a bit delayed and is not evident with just 20 samples.

# Plot (Linear Regression) - Poisson

<!--
<tr>
<td> <img src="src/cross_gen_gap/graphics/poisson_smaller_e_paper.png" alt="Loss values for different training sizes and epsilon values(Paper)" style="width: 500px;"/> </td>
<td> <img src="src/cross_gen_gap/graphics/poisson_smaller_e.png" alt="Loss values for different training sizes and epsilon values" style="width: 380px;"/> </td>
</tr>
-->

Loss from the paper | Our loss values
-|-
![Loss](src/cross_gen_gap/graphics/poisson_smaller_e_paper.png) | ![Loss](src/cross_gen_gap/graphics/poisson_smaller_e.png)

The above graphs show the result for Linear Regression with data sampled from the Poisson distribution.
The loss for the standard model($\varepsilon$ = 0) shows a consistent behavior in both the graphs. However, for other values of $\varepsilon$, we see that the behavior diverges from the paper. The test loss in the reproduced graph shows an initial upward trend followed by convergence at values much higher than the ones in the plot from the original paper. This will be investigated in the next experiments we will conduct.

### Conclusion

The paper points out that depending on the strength of the adversary, it will not always help to increase the size of the training dataset to adversarially train models and obtain a good performance on natural/ benign test data. This main trend is also visible in our own experiments. In scenarios with a medium adversary, our results are mory blurry due to the limited amount of repetitions of the experiments. 

### Possible Extensions

The data's empirical studies focus on the one-dimensional case for Binary Classification and Linear Regression. A first step will be to extend this to higher-dimensional data and then go to more complex models and see how they differ. Also, different approaches like FGSM and PGD will be interesting to compare in this regard.

## Second paper: Adversarial Robustness vs. Model Compression, or Both? - *Shaokai Ye, Kaidi Xu, Sijia Liu, Jan-Henrik Lambrechts, Huan Zhang, Aojun Zhou, Kaisheng Ma, Yanzhi Wang, Xue Lin*

The second paper tackles the dilemma od adversarial traning where achieving adversarial robustness requires significant capacity of the network in order to achieve competative performance. This may limit its use for security-critical scenarios especially in resource constrained application systems. In order to address this issue, the authors propose a framework of concurrent adversarial training and weight pruning that enables model compression while still preserving the adversarial robustness. 

### General setting

Alternating direction method of multipliers (ADMM) allows for concurrent adversarial training and weight pruning. This framework is built on the augmented Lagrangian of an equality constrained problem. Consequently, the problem is decompossed into two subproblems that are solved iteratively.
It is used in combination with **filter pruning scheme**, which is a common pruning choice to 
facilitate the implementation of sparse neural networks on hardware.

### Experimental setting

In Table1, we present LeNet network tested in the paper with its model architectures specified by the width scaling factor $\omega$. By using different $\omega$ values one can obtain adversarial baselines of different complexities.

$$\text {LeNet} | 2 * \omega, 4* \omega, FC (196*\omega, 64*\omega), FC (64*\omega,10) $$

Table1. Network stucture used in original experiments where $FC$ means fully coonected layer.
Other numbers denote the numbers of filters in convolutional layers. Here $\omega$ is the scaling factor of the LeNet network. Each layer is equally scaled with $ \omega $.

The experimental pipeline is summarized in the flowchart bellow.
 After obtaining an $\omega$ specific adversarial baseline a compression via weight pruning is applied. The compression rate is controled with the prune ratios. These ratios are only defined for the first two convolutional layers. In context of filter fruning scheme this means we prune 80% and 94.7% of weights in first and second convolutional layer respectively. 

Note: The prune ratio values of 0.8 and 0.947 are for illustration purposes. For the matter of fact, these values were predifined in the paper repo's config file withouht any further explanation from the authors side.    
We use these values as a starting point in our own experiments to closely examine their relationship to the compression rate.



![summary of the approach](src/model_compression/graphics/summary_approach.png)

By leveraging the ADMM approach, it turns out that the model obtained after the compression step preserves adversarial robustness compared to the starting baseline. This can be seen from Table2 which summarizes original paper results carried out on MNIST. 

For $\omega$=16 adversarial baseline (99.02%/94.65%) one can obtain a pruned model with the target size of $\omega$=8 via compression rate of 2 (99.01%/95.44%) or even much smaller model of size $\omega$=1 via stronger compression rate of 16 (96.19%/87.79%) while still achieving competitive natural test accuracy/adversarial test accuracy. We can notice that this behavior is present when going from small to large compression rate across all $\omega$ (red arrow).

![MNIST Experiment](src/model_compression/graphics/mnist_experiment_summary.png)

 Table2. Natural test/adversarial test accuracy on MNIST of naturally trained model with different size $\omega$	(second column),  
 adversarially trained model with different size $ \omega$ (third column), concurrent adversarial training and weight pruning from a large size to a small size compression rate 


### Results and Discussion

In this section we present our own experimental results on MNIST dataset using LeNet architecture specified with Table1. We report natural test/adversarial test accuracy for subset of $\omega$ that is 1, 2, 4 and 16. Before we present our findings on concurrent adversarial training and weight pruning we shortly showcase obtained results for natural and adversarial baseline across above-mentioned $\omega$ values (Table3). We note that there is no significant deviation between our experiment and original results.

![MNIST Experiment](src/model_compression/graphics/experiment_vanilla.png)
Table3. Validating natural and adversarial baseline test accuracies across target $\omega$ values


In our first ADMM experiment we direct our focus towards predifined prune ratio values and examining 
how do they influence the compression rate acrross different $\omega$ (Table4). 

Main observations:

*  with given prune ratios we achieve almost ~16x times compression rate (15.407) for $\omega$=16, 
meaning for starting $\omega$=16 the size after pruning should be close to $\omega$=1 and when we 
compare achieved natural test/adversarial test accuracy to corresponding one from the paper we see they 
are indeed in a really close range (our experiment: 97.26/87.94 paper: 96.19/87.79)
 
*  according to the paper supposed/"allowed" compression rate for $\omega$=4 should be 4 and 2 but we 
got ~14x compression rate (13.6); same holds for $\omega$=2 where only possible compression rate should 
be 2 but we obtained ~7x compression rate

Main conclusion:

* high prune ratio values lead to high compression rates across different $\omega$ which gives us a 
search direction for further analysis 

Next, we continue with our experiments in order to demistify second observation. 

<!--img src="src/model_compression/graphics/experiment_I.png" align-center width="900"/-->
![MNIST Experiment](src/model_compression/graphics/experiment_I.png)
Table4. Influence of conv1: 0.8 and conv2: 0.947 prune ratio


In our second experiment we investigate influence of different prune ratio values for fixed $\omega$. Here we define two settings, first one with the default prune ratio and second setting with its halved version (Table5).

Main observation:
* with setting II we obtain compression rate of 2x whose output matches one from the Table2 (our experiment 11.35/11.35 paper: 11.35/11.35)

Main conclusion:
* different prune ratios lead to different compression rates but same output in terms of natural test/advesarial test accuracy (for $\omega$=2)

In order to check whether this conclusion holds in general we conduct experiment under the same settings but for $\omega$=16

<!--img src="src/model_compression/graphics/experiment_II.png" align-center width="900"/-->
![MNIST Experiment](src/model_compression/graphics/experiment_II.png)
Table5. Influence of different prune ratios for $ \omega  = 2$ 


By analysing Table6 we can see that for setting II we obtain ~2x compression rate (1.87), meaning for starting $\omega$=16 the size after the pruning should be close to $\omega$=8 and when we compare achieved test accuracies with ones from the Table2 we indeed see that they almost exactly match. 
(exp: 99.01/93.23 paper: 99.01/95.44)

<!--img src="src/model_compression/graphics/experiment_III.png" align-center width="900"/-->
![MNIST Experiment](src/model_compression/graphics/experiment_III.png)
Table6. Influence of different prune ratios for } $\omega = 16$


We further continue with exploration of prune ratio space and repeat previous experiment but with the aditional setting, that is we wanted to observe what kind of change brings prune ratio value inbetween two previous ones (Table7). 

Main observation:

* by taking 75% of the default prune ratio we obtain ~3.3 compression rate 98.79/93.30 ; the closest value from the paper is 4x compression rate that is 98.87/94.77
* by decreasing the value of prune ratios (I->II->III) we achieve smaller compression rate which is consistent with the main premise of the paper (reference to the red arrow from the Table2)

Main conclusion:

* eventhough we didn't achieve exact 16x, 4x and 2x compression rates, results are still comperable <br>
(our experiment: 97.26/87.94 for 15.4x; paper: 96.19/87.79 for 16x  <br>
 our experiment: 98.79/93.30 for 3.34x; paper: 98.87/94.77 for 4x <br>
 our experiment: 99.01/93.23 for 1.87x; paper: 99.01/95.44 for 2x)
 
In the following experiment we took prune ratio values inbetween setting I and setting II from Table7 in order to validate results for 8x compression rate

<!--img src="src/model_compression/graphics/experiment_IV.png" align-center width="900"/-->
![MNIST Experiment](src/model_compression/graphics/experiment_IV.png)
Table7. Influence of three different prune ratios for $\omega = 16 $ 


Main observations from fifth experiment (Table 8):

* by taking the prune ratio inbetween the default one and its 3/4 version (setting II) we achieve 5.7x compression rate (98.62/94.07) which is not exactly the same as 8x compression rate (98.07/89.95) but it follows the main premise of the paper meaning if we would increase the prune ratio values we would get closser to 8x

Main conclusion:

* although we didn't achieve exact 16x, 8x, 4x and 2x compression rates, results are still comperable 

By closely examining these four different prune ratio settings we managed to obtain rough borders of search space for particular compression rate. To validate this, we conduct additional experiment for $\omega$=4 with the goal to achieve 4x and 2x compression rates. For the former one we narrow down our search inbetween setting II and setting III and for later one inbetween setting III and stting IV. Results are presented in Table9.

<!--img src="src/model_compression/graphics/experiment_V.png" align-center width="900"/-->
![MNIST Experiment](src/model_compression/graphics/experiment_V.png)
Table8. Influence of four different prune ratios for $\omega = 16$


Main observations from sixt experiment (Table9):

* with setting I we achieve ~4x times compression rate (3.88) for $\omega$=4, meaning for starting $\omega$=4 the size after pruning is close to $\omega$=1
(our experiment: 96.28/91.34 paper: 96.22/89.41)

* with setting II we achieve ~2x times compression rate (2.26) for $\omega$=4, meaning for starting $\omega$=4 the size after pruning is $\omega$=2
(our experiment: 97.30/90.95 paper: 97.68/91.77)

Main conclusion:

* by adopting search guidelines from the fifth experiment we managed to validate findings for $\omega$=4 as well

<!--img src="src/model_compression/graphics/experiment_VI.png" align-center width="900"/-->
![MNIST Experiment](src/model_compression/graphics/experiment_VI.png)
Table9. Influence of different prune ratios for $\omega = 4$


General Note: All conducted experiments can be repeated by running coresponding call from run.sh.example file (`/src/model_compression/Robustness-Aware-Pruning-ADMM/ADMM_examples/mnist`)

### Conclusion

In summary, the value of weight pruning is essential in the adversarial training setting: it is possible to acquire a network of small model size (by weight pruning) with both high natural test accuracy and adversarial test accuracy. We saw that for smaller values of prune ratios we obtain smaller compression rates and vice versa i.e. there is a positive correlation between prune ratios and compression rate accross different $\omega$.

### Possible directions of paper extension 

* What is the ratio between their ADMM approach and just using adversarial training once
and pruning afterwards? <br>
* Use different pruning schemes and see how this compares <br>
* Pruning is only done for conv layers, what about other models/ datasets/ layers?